In [48]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
import sys;
sys.path.append(r'C:\Monitor\\')
sys.path.append(r'\\ac-geneva-24\E\grains trading\Streamlit\Monitor\\')

from datetime import datetime as dt
from copy import deepcopy
import numpy as np
import pandas as pd

import Utilities.Modeling as um
import Utilities.Weather as uw
import Utilities.GLOBAL as GV
import Utilities.Utilities as uu
import statsmodels.api as sm

import Models.Soybean_USA_Yield_GA as sy

import APIs.Bloomberg as ba
from sklearn.model_selection import TimeSeriesSplit

import warnings # supress warnings
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None

In [65]:
scope = sy.Define_Scope()
raw_data = sy.Get_Data_All_Parallel(scope) 

multi_ww_dt_s=dt(2022,5,1)
multi_ww_dt_e=dt(2022,9,1)

multi_ww_freq_start='1D'
multi_ww_freq_end='1D'

multi_ww_ref_year_s=dt(2022,1,1)
    
raw_data['multi_ww_df']=um.generate_weather_windows_df(raw_data['w_w_df_all']['hist'], date_start=multi_ww_dt_s, date_end=multi_ww_dt_e, ref_year_start=multi_ww_ref_year_s, freq_start=multi_ww_freq_start, freq_end=multi_ww_freq_end)


100%|██████████| 124/124 [00:19<00:00,  6.21it/s]


In [66]:
model_df = pd.concat([raw_data['yield'],raw_data['multi_ww_df']], sort=True, axis=1, join='inner')
model_df=model_df.dropna() # Needed because (maybe) the current year has some windows that have not started yet

# Train Test Splits 
min_train_size= min(10,len(model_df)-3)
folds_expanding = TimeSeriesSplit(n_splits=len(model_df)-min_train_size, max_train_size=0, test_size=1)
folds = []
folds = folds + list(folds_expanding.split(model_df))

In [67]:
y_col  ='Yield'
y_df = model_df[[y_col]]

In [68]:
X_df = deepcopy(model_df)

cols=list(m.params.index)
cols.remove('const')
X_df=X_df[cols]


cv_score = um.stats_model_cross_validate(X_df, y_df, folds)

In [69]:
np.mean(cv_score['cv_p'])

0.18645578887790115

In [70]:
X_df.head()

,year,USA_TempMax_Aug21-Aug28,USA_Sdd30_Jun17-Aug19,USA_TempMax_Aug16-Aug19,USA_Sdd30_Jul04-Aug20,USA_Sdd30_Jun27-Jul15
year,,,,,,
1985,1985,24.624972,0.535132,26.473488,0.628962,1.012893
1986,1986,26.659765,0.921108,29.105613,0.923356,0.865736
1987,1987,24.709041,1.342054,30.206546,1.565715,0.324604
1988,1988,27.760942,2.973521,35.356721,3.018119,2.246986
1989,1989,28.301939,0.903515,26.984517,0.970026,1.707921


In [40]:
np.mean(raw_data['w_w_df_all']['hist'][['USA_TempMax']].loc['1985-08-20':'1985-08-27'])

USA_TempMax    24.134947
dtype: float64

In [31]:
model_df.head()

,USA_TempMax_Aug21-Aug28,USA_Sdd30_Jun17-Aug19,USA_TempMax_Aug16-Aug19,USA_Sdd30_Jul04-Aug20,USA_Sdd30_Jun27-Jul15,year
year,,,,,,
1985,24.624972,0.535132,26.473488,0.628962,1.012893,1985
1986,26.659765,0.921108,29.105613,0.923356,0.865736,1986
1987,24.709041,1.342054,30.206546,1.565715,0.324604,1987
1988,27.760942,2.973521,35.356721,3.018119,2.246986,1988
1989,28.301939,0.903515,26.984517,0.970026,1.707921,1989


In [25]:
m = um.pick_model('GA_soy',72)
ww = um.from_cols_to_var_windows(m.params.index)

Start Deserializing: GA_soy
Done Deserializing: GA_soy


In [30]:
model_df = um.extract_yearly_ww_variables(w_df = raw_data['w_w_df_all']['hist'],var_windows= ww)
model_df['year']=model_df.index

In [38]:
np.mean(raw_data['w_w_df_all']['hist'][['USA_TempMax']].loc['1985-08-21':'1985-08-28'])

USA_TempMax    24.624972
dtype: float64

In [42]:
y_df = raw_data['yield'][raw_data['yield'].index<2022][['Yield']]

In [43]:
X_df=model_df

folds = um.folds_expanding(X_df, min_train_size=10)
# um.print_folds(folds = folds, years=train_df.index.values)

# Cross-Validation calculation
cv_score = um.stats_model_cross_validate(X_df, y_df, folds)

In [5]:
m.params

const                     -960.550291
year                         0.509614
USA_TempMax_Aug21-Aug28     -0.383391
USA_Sdd30_Jun17-Aug19        1.804934
USA_TempMax_Aug16-Aug19     -0.348013
USA_Sdd30_Jul04-Aug20       -0.909144
USA_Sdd30_Jun27-Jul15        0.800158
dtype: float64

In [44]:
np.mean(cv_score['cv_p'])

0.1864557888779011

In [25]:
f='GA_soy'
dm_best = {}
dm_best[f] = uu.deserialize(f)
r=dm_best[f]

r.keys()

Start Deserializing: GA_soy
Done Deserializing: GA_soy


dict_keys(['best_fitness', 'model', 'MAE', 'MAPE', 'fitness', 'corr', 'cv_p', 'cv_r_sq', 'cv_MAE', 'cv_MAPE', 'cv_corr'])

In [37]:
np.mean(r['cv_p'][72])

0.001679548798393492

In [ ]:
print(len(r['model']))
print(len(r['corr']))
print(r['cv_p'][0])